# 02: Lexical Similarity Analysis

This notebook demonstrates **lexical similarity** - a weak signal based on token-level text comparison using TF-IDF.

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.similarity.lexical import LexicalSimilarity
from src.io import load_submissions
from src.normalization import get_normalizer

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Basic Lexical Similarity

In [ ]:
# Initialize analyzer
lexical_analyzer = LexicalSimilarity()

#Test codes
code1 = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)
'''

code2 = '''
def fact(x):
    if x <= 1:
        return 1
    return x * fact(x-1)
'''

# Compute similarity
similarity = lexical_analyzer.compute_similarity(code1, code2)

print("📊 Lexical Similarity Test")
print("=" * 60)
print(f"Similarity Score: {similarity:.1f}%")
print("=" * 60)

# Token statistics
stats1 = lexical_analyzer.get_token_statistics(code1)
stats2 = lexical_analyzer.get_token_statistics(code2)

print("\nCode 1 Statistics:")
for key, value in stats1.items():
    print(f"  {key}: {value}")

print("\nCode 2 Statistics:")
for key, value in stats2.items():
    print(f"  {key}: {value}")

## 2. Pairwise Similarity Matrix

Compute lexical similarity for all submission pairs.

In [ ]:
# Load submissions
submissions = load_submissions('../data/raw/sample_submissions.csv')

# Normalize all codes first
normalizer = get_normalizer('python')
normalized_codes = []
for sub in submissions:
    normalizer.reset_counters()
    normalized_codes.append(normalizer.normalize(sub['code']))

# Compute pairwise similarities
similarity_matrix = lexical_analyzer.compute_pairwise_similarities(normalized_codes)

print(f"Computed {len(submissions)}x{len(submissions)} similarity matrix")
print(f"Shape: {similarity_matrix.shape}")

## 3. Similarity Heatmap

In [ ]:
# Create heatmap
submission_ids = [sub['submission_id'] for sub in submissions]

plt.figure(figsize=(10, 8))
ax = sns.heatmap(similarity_matrix, 
                 annot=True, 
                 fmt='.1f',
                 cmap='RdYlGn',
                 vmin=0, 
                 vmax=100,
                 xticklabels=submission_ids,
                 yticklabels=submission_ids,
                 cbar_kws={'label': 'Similarity (%)'},
                 linewidths=0.5,
                 linecolor='gray')

plt.title('Lexical Similarity Heatmap (Normalized Code)', 
          fontsize=16, fontweight='bold', pad=20)
plt.xlabel('Submission ID', fontsize=12, fontweight='bold')
plt.ylabel('Submission ID', fontsize=12, fontweight='bold')

# Rotate labels
plt.xticks(rotation=45)
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()

# Find highest similarity pairs
print("\n🔝 Top 3 Similar Pairs (excluding self):")
print("=" * 60)

# Get upper triangle (excluding diagonal)
pairs = []
n = len(submissions)
for i in range(n):
    for j in range(i+1, n):
        pairs.append((submission_ids[i], submission_ids[j], similarity_matrix[i][j]))

# Sort by similarity
pairs.sort(key=lambda x: x[2], reverse=True)

for i, (id1, id2, sim) in enumerate(pairs[:3], 1):
    print(f"{i}. {id1} ↔ {id2}: {sim:.1f}%")

## 4. Why Lexical Similarity is a Weak Signal

Let's see why we can't rely on lexical similarity alone.

In [ ]:
# Same algorithm, very different tokens
fibonacci_recursive = '''
def fib(n):
    if n <= 1:
        return n
    return fib(n-1) + fib(n-2)
'''

fibonacci_iterative = '''
def fibonacci(num):
    a, b = 0, 1
    for i in range(num):
        a, b = b, a + b
    return a
'''

# Different algorithm, similar tokens
factorial = '''
def fib(n):
    if n <= 1:
        return 1
    return n * fib(n-1)
'''

# Test 1: Same algorithm, different style
sim1 = lexical_analyzer.compute_similarity(fibonacci_recursive, fibonacci_iterative)

# Test 2: Different algorithm, similar tokens
sim2 = lexical_analyzer.compute_similarity(fibonacci_recursive, factorial)

print("⚠️  Lexical Similarity Weaknesses")
print("=" * 70)
print(f"\nTest 1: Fibonacci (recursive) vs Fibonacci (iterative)")
print(f"  → Same algorithm, different implementation")
print(f"  → Lexical similarity: {sim1:.1f}%")
print(f"  → ❌ LOW score despite being the SAME algorithm!")

print(f"\nTest 2: Fibonacci (recursive) vs Factorial (recursive)")
print(f"  → Different algorithms, similar structure")
print(f"  → Lexical similarity: {sim2:.1f}%")
print(f"  → ⚠️  Could be misleadingly high!")

print("\n" + "=" * 70)
print("\n💡 KEY TAKEAWAY:")
print("   Lexical similarity can MISS real plagiarism (refactored code)")
print("   and give FALSE POSITIVES (similar syntax, different logic).")
print("\n   This is why we need STRUCTURAL and SEMANTIC signals!")

## 5. Comparison: Before vs After Normalization

In [ ]:
# Test with disguised code
original = '''
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n-1)
'''

disguised = '''
def compute_factorial(number):
    # Base case
    if number <= 1:
        return 1
    # Recursive multiplication
    result = number * compute_factorial(number - 1)
    return result
'''

# Before normalization
sim_before = lexical_analyzer.compute_similarity(original, disguised)

# After normalization
normalizer.reset_counters()
norm_original = normalizer.normalize(original)
normalizer.reset_counters()
norm_disguised = normalizer.normalize(disguised)

sim_after = lexical_analyzer.compute_similarity(norm_original, norm_disguised)

# Visualize improvement
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['Before\nNormalization', 'After\nNormalization']
scores = [sim_before, sim_after]
colors = ['#e74c3c', '#2ecc71']

bars = ax.bar(categories, scores, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels
for bar, score in zip(bars, scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{score:.1f}%',
            ha='center', va='bottom', fontsize=14, fontweight='bold')

ax.set_ylabel('Lexical Similarity (%)', fontsize=12, fontweight='bold')
ax.set_title('Impact of Normalization on Lexical Similarity\n(Same Algorithm, Disguised Code)', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_ylim(0, 110)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

improvement = sim_after - sim_before
print(f"\n📈 Similarity increase after normalization: +{improvement:.1f}%")
print(f"   Normalization helps reveal hidden similarities!")

## Summary

✅ **Fast Computation**: TF-IDF is very efficient for quick screening

✅ **Supporting Evidence**: Good for confirming other signals

❌ **Weak Alone**: Cannot reliably detect algorithmic plagiarism

❌ **Fooled by Refactoring**: Low scores for identical algorithms with different style

⚠️ **False Positives**: Similar syntax doesn't mean same logic

**Key Insight**: Lexical similarity gets only **15% weight** in our system because it's easily fooled. We need structural and semantic analysis for robust detection!

**Next Steps**: Proceed to notebook 03 to see the REAL power - AST vs RK-GST structural similarity!